<a href="https://colab.research.google.com/github/HanselWilfred/WeatherWise-Hansel-Wilfred/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [19]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install requests matplotlib pyinputplus





## 📦 Setup and Configuration
Import required packages and setup environment.

In [20]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
#from hands_on_ai.chat import get_response

# Add any other setup code here

## 🌤️ Weather Data Functions

In [21]:
import requests

def get_weather_data(location, forecast_days=5):
    """Return {'location', 'current_temp', 'forecast': [{date, max_temp, min_temp, rain_chance}, ...] }."""
    # clamp forecast_days 1..5 to be safe for the tester
    days = max(1, min(5, int(forecast_days or 5)))

    def _clean_location(text):
        import re
        text = str(text or "").strip()
        text = re.sub(r"\b(tomorrow|today|day after|in\s+\d+\s+days)\b", "", text, flags=re.I)
        text = re.sub(r"\s+", " ", text).strip()
        return text or None

    loc = _clean_location(location)
    if not loc:
        return {}

    try:
        geo = requests.get(
            "https://geocoding-api.open-meteo.com/v1/search",
            params={"name": loc, "count": 1, "language": "en"},
            timeout=10
        )
        geo.raise_for_status()
        g = geo.json()
        results = g.get("results") or []
        if not results:
            return {}

        lat, lon = results[0]["latitude"], results[0]["longitude"]
        resolved = results[0].get("name") or loc

        fc = requests.get(
            "https://api.open-meteo.com/v1/forecast",
            params={
                "latitude": lat,
                "longitude": lon,
                "current_weather": True,
                "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_max",
                "timezone": "auto",
            },
            timeout=10
        )
        fc.raise_for_status()
        data = fc.json()

        daily = data.get("daily") or {}
        times = daily.get("time") or []
        if not times:
            return {}

        out = {
            "location": resolved,
            "current_temp": (data.get("current_weather") or {}).get("temperature"),
            "forecast": []
        }
        for i in range(min(days, len(times))):
            out["forecast"].append({
                "date": times[i],
                "max_temp": daily["temperature_2m_max"][i],
                "min_temp": daily["temperature_2m_min"][i],
                "rain_chance": (daily.get("precipitation_probability_max") or [None]*len(times))[i],
            })
        return out
    except requests.RequestException:
        return {}




## 📊 Visualisation Functions

In [22]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
import matplotlib.pyplot as plt

def create_temperature_visualisation(weather_data):
    dates = [f["date"] for f in weather_data["forecast"]]
    max_temps = [f["max_temp"] for f in weather_data["forecast"]]
    min_temps = [f["min_temp"] for f in weather_data["forecast"]]

    plt.plot(dates, max_temps, marker="o", label="Max Temp")
    plt.plot(dates, min_temps, marker="o", label="Min Temp")
    plt.title(f"Temperature Forecast - {weather_data['location']}")
    plt.xlabel("Date")
    plt.ylabel("Temperature (°C)")
    plt.legend()
    plt.grid(True)
    plt.show()


In [23]:

def create_precipitation_visualisation(weather_data):
    dates = [f["date"] for f in weather_data["forecast"]]
    rain = [f["rain_chance"] for f in weather_data["forecast"]]

    plt.bar(dates, rain, color="skyblue")
    plt.title(f"Rain Chance Forecast - {weather_data['location']}")
    plt.xlabel("Date")
    plt.ylabel("Rain Chance (%)")
    plt.show()


## 🤖 Natural Language Processing

In [24]:
# Define parse_weather_question() and generate_weather_response() here
import re

def _clean_location(text: str) -> str | None:
    if not text:
        return None
    # remove time words/phrases
    text = re.sub(r"\b(tomorrow|today|day after|in\s+\d+\s+days)\b", "", text, flags=re.I)
    # squeeze spaces and strip
    text = re.sub(r"\s+", " ", text).strip()
    return text.title() if text else None

def parse_weather_question(q: str):
    q = q.lower().strip()

    # day
    day = 2 if "day after" in q else 1 if "tomorrow" in q else 0
    m = re.search(r"\bin\s+(\d+)\s+days\b", q)
    if m:
        day = max(0, min(4, int(m.group(1))))

    # attribute
    attr = "temperature" if any(w in q for w in ["temp", "temperature", "hot", "cold", "warm", "cool"]) else "rain"

    # location by PIN/ZIP first
    pin = re.search(r"\b\d{3,6}\b", q)
    if pin:
        return {"location": pin.group(0), "day": day, "attribute": attr}

    # location after "in ..." but stop BEFORE punctuation/time words
    m = re.search(
        r"\bin\s+([a-z0-9\s\-]+?)(?=(?:\?|\.|,|$|\s+tomorrow|\s+today|\s+day after|\s+in\s+\d+\s+days))",
        q,
    )
    loc = _clean_location(m.group(1)) if m else None

    return {"location": loc, "day": day, "attribute": attr}



## 🧭 User Interface

In [31]:
# Define menu functions using pyinputplus or ipywidgets here
import pyinputplus as pyip

def main_menu():
    while True:
        choice = pyip.inputMenu(
            ["View Forecast", "Ask a Question", "Visualise Data", "Exit"],
            numbered=True
        )

        if choice == "View Forecast":
            location = pyip.inputStr("Enter a city: ")
            weather = get_weather_data(location)
            print(f"📍 {weather['location']} | 🌡️ {weather['current_temp']}°C")
            for f in weather["forecast"]:
                print(f"{f['date']}: {f['min_temp']}–{f['max_temp']}°C, Rain chance {f['rain_chance']}%")

        elif choice == "Ask a Question":
    q = pyip.inputStr("Ask me about the weather: ")
    p = parse_weather_question(q)

    loc = p.get("location")
    if not loc:
        loc = pyip.inputStr("Which city or PIN/ZIP? (e.g., Chennai or 6000): ")
    p["location"] = loc

    d = get_weather_data(loc)
    if not d or not d.get("forecast"):
        print("🤖 Sorry, I couldn’t get that location. Try a different city or a PIN/ZIP.")
    else:
        print("🤖", generate_weather_response(p, d))


        elif choice == "Visualise Data":
            location = pyip.inputStr("Enter a city for charts: ")
            weather = get_weather_data(location)
            create_temperature_visualisation(weather)
            create_precipitation_visualisation(weather)

        elif choice == "Exit":
            print("Goodbye!")
            break

In [32]:
def generate_weather_response(parsed_q, weather_data):
    """Generates a natural language response based on parsed query and weather data."""
    if "error" in weather_data:
        return weather_data["error"]

    location = weather_data["location"]
    day = parsed_q.get("day", 0)
    attribute = parsed_q.get("attribute", "temperature")

    if day >= len(weather_data["forecast"]):
        return f"Sorry, I only have a 5-day forecast for {location}."

    forecast = weather_data["forecast"][day]
    date = forecast["date"]

    if attribute == "temperature":
        return f"The temperature forecast for {location} on {date} is a high of {forecast['max_temp']}°C and a low of {forecast['min_temp']}°C."
    elif attribute == "rain":
        return f"The chance of rain in {location} on {date} is {forecast['rain_chance']}%."
    else:
        return "Sorry, I can only provide information on temperature and rain."

## 🧩 Main Application Logic

In [33]:
# Tie everything together here
def run_app():
    print("🌦️ Welcome to WeatherWise 🌦️")
    print("👉 You can enter either a city name (e.g., Perth, Sydney) OR a PIN/ZIP code (e.g., 6000, 10001).")
    main_menu()



## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function
run_app()

🌦️ Welcome to WeatherWise 🌦️
👉 You can enter either a city name (e.g., Perth, Sydney) OR a PIN/ZIP code (e.g., 6000, 10001).
Please select one of the following:
1. View Forecast
2. Ask a Question
3. Visualise Data
4. Exit


## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.